![alt text](img/LM.png)
# Kurs: Deep Learning, Text Mining i XAI w Pythonie

## Autor: Piotr Ćwiakowski

### Lekcja 10. Sieci neuronowe w NLP 

### Spis treści

1. Word2Vec
2. Przykład aplikacyjny 

# 1 Word2vec

## Wprowadzenie
Reprezentacja słów (word embedding) jest wykonywana za pomocą modelu word2vec (skip-gram model). Polega na wytrenowaniu jednej prostej sieci neuronowej. Elementem sieci, który nas interesuje nie są jednak wartości dopasowane modelu, tylko wagi uzyskane w ukrytej warstwie. To one właśnie są wektorową reprezentacją słów, które potrzebujemy. Potrzebujemy zatem „fałszywego” zadania, żeby móc otrzymać wagi dla ukrytej warstwy. Tym zadaniem będzie przewidywanie prawdopodobieństwa wystąpienia w pobliżu danego słowa każdego innego słowa ze słownika. „Pobliże” jest często definiowane jako 5 słów przed i 5 słów po. Poniżej prosty przykład z 1 zdaniem i oknem -2/+2.

<img src="img/W1.png" width="70%">

Jak konkretnie trenowana jest sieć? Na podstawie wyprodukowanych z korpusu skipgramów tworzone są zestawy obserwacji (input, output), które następnie sekwencyjnie zasilają jedno warstwową MLP, która modeluje prawdopodobieństwo wystąpienia w kontekście słowa każdego z pozostałych wyrazów. Na przykładzie poniżej dla korpusu 10000 słów szacujemy sieć złożoną z 300 neuronów. Warstwa ukryta nie ma funkcji aktywacyjnej, ale końcowa warstwa używa funkcji softmax.

<img src="img/W2.png" width="50%">

Skupmy się teraz na warstwie ukrytej z przykładu z poprzedniego slajdu. W matematycznym sensie, redukuje ona liczbę wymiarów do 300. Wartościami w tych wymiarach są wartości uzyskane dla każdego słowa w warstwie ukrytej. Przyjrzyjmy się ilustracji:

<img src="img/W3.png" width="30%">

## 2. Przykład aplikacyjny - IMDB

In [4]:
# Wczytanie pakietów
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

In [5]:
# Przygotowanie danych
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [6]:
# Stworzenie modelu
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 flatten (Flatten)           (None, 16000)             0         
                                                                 
 dense (Dense)               (None, 250)               4000250   
                                                                 
 dense_1 (Dense)             (None, 1)                 251       
                                                                 
Total params: 4,160,501
Trainable params: 4,160,501
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Estymacja
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128, verbose=2)

Epoch 1/2
196/196 - 6s - loss: 0.4430 - accuracy: 0.7719 - val_loss: 0.2926 - val_accuracy: 0.8765 - 6s/epoch - 31ms/step
Epoch 2/2
196/196 - 6s - loss: 0.1689 - accuracy: 0.9365 - val_loss: 0.3166 - val_accuracy: 0.8732 - 6s/epoch - 31ms/step


In [8]:
# Ewaluacja
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.32%


# 3. Przykład aplikacyjny - word2vec w sieciach RNN

### Pakiety

In [2]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

### Wczytanie i przygotowanie danych

In [10]:
train_data = text_dataset_from_directory("./data/movie-reviews/train")
test_data = text_dataset_from_directory("./data/movie-reviews/test")

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [11]:
def prepareData(dir):
    data = text_dataset_from_directory(dir)
    return data.map(lambda text, label: (regex_replace(text, '<br />', ' '), label))

train_data = prepareData('./data/movie-reviews/train')
test_data = prepareData('./data/movie-reviews/test')

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [12]:
for text_batch, label_batch in train_data.take(1):
    print(text_batch.numpy()[0])
    print(label_batch.numpy()[0]) # 0 = negative, 1 = positive

b'Let\'s begin with that theme song sung by Christopher Cross. The song is "If you get caught between the moon and New York City." It\'s a great theme and song even after all these years, it never gets tiring. It really is a great song about New York City as well. Anyway, the great Dudley Moore CBE stars as a spoiled drunken millionaire who is engaged to Jill Eikenberry\'s character in the film. Jill would later star on LA Law. Anyway, he is served by his wonderful British butler, Sir John Gielgud OM who won an Academy Award for his performance in the film as Best Supporting Actor. Arthur falls in love with Liza Minnelli\'s character who is perfect in this film besides her performance in her Oscar winning role in Cabaret. No, Liza doesn\'t get to sing. She plays a diner waitress. Anyway I love Geraldine Fitzgerald as the Bach matriarch of the family who decides the family\'s fortune. Anyway, she is fabulous and should have gotten an academy award nomination herself for Best Supporting 

### Architektura modelu

In [13]:
model = Sequential()
model.add(Input(shape=(1,), dtype="string"))

In [14]:
max_tokens = 1000
max_len = 100

vectorize_layer = TextVectorization(
  # Maksymalny rozmiar słownika - w ten sposób ograniczamy liczbę zmiennych (do najbardziej popularnych), a wszystkie pozostałe słowa są zaklasyfikowane do kolumny out-of-vocabulary (OOV) 
  max_tokens=max_tokens,
  # Wynikiem (outputem) będą liczby zamiast słów
  output_mode="int",
  # Maksymalna liczba słów w każdej obserwacji - jeśli w tekscie jest wiecej - obcinamy, jesli mniej - wydłużamy (o zera)
  output_sequence_length=max_len,
)

# Jeśli tworzymy obiekt ze standaryzowanym słownikiem (max_tokens) - musimy go wytrenować na zbiorze
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

# Dodanie warstwy wektoryzującej tekst, nota bene słów jest 1000 + 1 (ostatnie to OOV)
model.add(vectorize_layer)
model.add(Embedding(max_tokens + 1, 128))

# Wykańczamy sieć neuronami rekurencyjnymi i feed-forward
model.add(LSTM(64)) # warstwa rekurencyjna
model.add(Dense(64, activation="relu")) # warstwa feed-forward
model.add(Dense(1, activation="sigmoid")) # warstwa wyjściowa

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 100, 128)          128128    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 181,761
Trainable params: 181,761
Non-trainable params: 0
________________________________________________

### Kompilowanie modelu 

In [15]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

In [16]:
# Model trenuje się kilka minut
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 36s 44ms/step - loss: 0.5369 - accuracy: 0.7302
Epoch 2/10
782/782 [==============================] - 48s 61ms/step - loss: 0.4446 - accuracy: 0.7952
Epoch 3/10
782/782 [==============================] - 48s 61ms/step - loss: 0.4134 - accuracy: 0.8138
Epoch 4/10
782/782 [==============================] - 48s 61ms/step - loss: 0.3886 - accuracy: 0.8274
Epoch 5/10
782/782 [==============================] - 48s 62ms/step - loss: 0.3698 - accuracy: 0.8366
Epoch 6/10
782/782 [==============================] - 46s 59ms/step - loss: 0.3576 - accuracy: 0.8452
Epoch 7/10
782/782 [==============================] - 47s 60ms/step - loss: 0.3375 - accuracy: 0.8558
Epoch 8/10
782/782 [==============================] - 47s 60ms/step - loss: 0.3219 - accuracy: 0.8638
Epoch 9/10
782/782 [==============================] - 45s 58ms/step - loss: 0.3052 - accuracy: 0.8704
Epoch 10/10
782/782 [==============================] - 45s 58ms/step - loss: 0.293

In [17]:
# Should print a very high score like 0.98.
print(model.predict([
  "i loved it! highly recommend it to anyone and everyone looking for a great movie to watch.",
]))

# Should print a very low score like 0.01.
print(model.predict([
  "this was awful! i hated it so much, nobody should watch this. the acting was terrible, the music was terrible, overall it was just bad.",
]))

1/1 [==============================] - 1s 720ms/step
[[0.98735964]]
1/1 [==============================] - 0s 71ms/step
[[0.02483406]]
